In [1]:
import json
import GPy
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from os.path import join, splitext
from os import listdir

In [2]:
data_path = '/home/rodrigo/Documents/curvas-de-luz-EL4106/OGLE_trainingDB/'
sub_class = 'CEPH/1O_20'

In [3]:
lc_list = [splitext(file)[0] for file in sorted(listdir(join(data_path, sub_class))) if splitext(file)[1] == '.dat']

In [4]:
print(len(lc_list))
file = open('database.csv','a')
def smooth(x, window_len=11, window='hanning'):
    if window_len < 3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='valid')
    return y    

166


In [5]:
for lc_index in range(162,len(lc_list)):
    
    data = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.time'))
    period = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.dat'))
    mjd, mag_r, err_r, mag_b, err_b = data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4]
    """
    mjd_b = mjd
    mjd_r = mjd
    """
    mjd_b = mjd[err_b < 9.999]
    mjd_r = mjd[err_r < 9.999]
    mag_b = mag_b[err_b < 9.999]
    mag_r = mag_r[err_r < 9.999]
    err_b = err_b[err_b < 9.999]
    err_r = err_r[err_r < 9.999]

    mask = err_b < np.mean(err_b) + 3*np.std(err_b)
    mjd_b = mjd_b[mask]
    mag_b = mag_b[mask]
    err_b = err_b[mask]
    mask = err_r < np.mean(err_r) + 3*np.std(err_r)
    mjd_r = mjd_r[mask]
    mag_r = mag_r[mask]
    err_r = err_r[mask]
    
    phi = np.mod(mjd_b, period)/period
    I = np.argsort(phi)
    phi_rs = np.linspace(0, 1, num=200)
    mag_b_rs = np.interp(phi_rs, phi[I], mag_b[I])
    

    kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m = GPy.models.GPHeteroscedasticRegression(phi[:, np.newaxis], mag_b[:, np.newaxis], kernel)
    m['.*het_Gauss.variance'] = err_b[:, np.newaxis]**2 
    m.het_Gauss.variance.fix() 
    #m.het_Gauss.period.fix() 
    m.optimize()

    m.optimize(messages=True)
    m.optimize_restarts(num_restarts = 5)
    mu, var = m._raw_predict(phi_rs[:, np.newaxis])
    # File writer
    file.write(json.dumps(phi_rs.tolist())+";"+json.dumps(mu[:,0].tolist())+";"+sub_class+'\n')
    print(lc_index)
file.close()
    

    


Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0002   3.337541e+02   9.362233e+03 
    01s79  0022   2.934833e+02   8.053788e-04 
Runtime:     01s79
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = 293.4832796545429
Optimization restart 2/5, f = 293.48327321179613
Optimization restart 3/5, f = 293.4832740156876
Optimization restart 4/5, f = 293.4832747018045
Optimization restart 5/5, f = 316.3418173329733
162
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0001   6.084287e+01   1.750557e-08 
    02s28  0022   4.715255e+01   1.416428e-06 
    05s03  0059   4.701861e+01   2.223831e-07 
Runtime:     05s03
Optimization status: Converged

Optimization restart 1/5, f = 47.01860722433912
Optimization restart 2/5, f = 60.84287446589542
Optimization restart 3/5, f = 60.84287234021485
Optimization restart 4/5, f = 60.84287145345934
Optimization 

In [6]:
print("Finished")

Finished
